Probando otra funcion de resumen que devuelva fechas de origen de la data.

In [3]:
from langchain.document_loaders import UnstructuredFileLoader
loader = UnstructuredFileLoader("../../data/mastercard/s3.pdf")
docs = loader.load()
context = docs[0].page_content

In [4]:
import openai
import os
import json

openai.api_key = os.environ["OPENAI_API_KEY"]
MODEL = "gpt-4-1106-preview"


def summarize_and_extract_metadata(text_to_summarize: str):
    """Summarize text using OpenAI's API."""
    function_description = f"""
    The user will give you an important document with some meaningful data surrounded by meaningless data.
    Extract all meaningful information from the document and create a summary, put that summary in the summary field.
    Then extract the name of the user and put it in the user field.
    Then extract the date of the document and put it in the date field.
    No important details should be left out.
    This is very important information for the user, so please be sure to include all the relevant information.
    Use this format:
    'The user [did/bought/used/paid/...] [something] on [date] at [place/store] for [amount of money].'
    """
    functions = [
        {
            "name": "default",
            "description": function_description,
            "parameters": {
                "type": "object",
                "properties": {
                    "summary": {
                        "type": "string",
                        "description": "The summary of the document according to the function description.",
                    },
                    "user": {
                        "type": "string",
                        "description": "[User's first name] [User's last name]",
                    },
                    "date": {
                        "type": "string",
                        "description": "Date in the format YYYY-MM-DD",
                    },
                },
            },
        }
    ]
    messages = [
        {"role": "user", "content": text_to_summarize},
    ]
    try:
        completion = openai.ChatCompletion.create(
            model=MODEL,
            messages=messages,
            functions=functions,
            function_call={"name": "default"},
        )
        arguments = json.loads(
            str(completion.choices[0]["message"]["function_call"]["arguments"])
        )

        return arguments

    except openai.error.InvalidRequestError:
        print("openai request failed")
        return None

In [5]:
summary = summarize_and_extract_metadata(context)

  Si guardo todo el documento despues puedo mandar este solo como contexto porque toma casi 4k tokens, en cambio si guardo los resumenes filtro solo el contenido util sin perder la referencia al archivo fuente por si se necesita.
  
  `"usage": {
    "prompt_tokens": 3803,
    "completion_tokens": 254,
    "total_tokens": 4057
  }`

In [24]:
type(summary)

dict

In [6]:
summary['date']


'2023-10-17'

In [7]:
summary['summary']

'The user Alexander Ditzend made purchases on various dates at Scape Park, Autopistas, Don Web, PPRO *Adobe, and other vendors on his Mastercard Platinum Credit Card, resulting in a total amount due of 102,041.04 ARS as of 17-Oct-23.'

In [33]:
summary['user']

'ALEXANDER DITZEND'

Paso siguiente: guardar el resumen en un indice junto con 'user' y 'date' como metadata ademas de 'source'. Que eso este disponible para hacer QA y generar preguntas de prueba con gpt4 para evaluar la capacidad.